## 설치

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.2 MB/s 
     |████████████████████████████████| 7.6 MB 45.4 MB/s 
     |████████████████████████████████| 163 kB 65.1 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 5.2 MB/s 
     |████████████████████████████████| 115 kB 47.2 MB/s 
     |████████████████████████████████| 212 kB 58.6 MB/s 
     |████████████████████████████████| 127 kB 40.9 MB/s 
     |████████████████████████████████| 115 kB 70.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 5.2 MB/s 
     |████████████████████████████████| 182 kB 55.3 MB/s 
     |████████████████████████████████| 166 kB 32.1 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 166 kB 46.3 MB/s 
     |████████████████████████████████| 162 kB 72.6 MB/s 
     |████████████████████████████████| 162 kB 62.2 MB/s 
     |████████████████████████████████| 158 kB 56.4 MB/s 
     |████████████████████████████████| 157 kB 68.8 MB/s 
     |████████████████████████████████| 157 kB 75.4 MB/s 
     |████████████████████████████████| 157 kB 55.1 MB/s 
     |████████████████████████████████| 157 kB 72.1 MB/s 
     |████████████████████████████████| 157 kB 68.9 MB/s 
     |████████████████████████████████| 157 kB 65.3 MB/s 
     |████████████████████████████████| 157 kB 68.2 MB/s 
     |████████████████████████████████| 156 kB 70.6 MB/s 


In [111]:
import torch
import torch.optim as optim

import numpy as np
import tqdm as tqdm

from transformers import(
    AutoModel,
    AutoModelWithLMHead,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    default_data_collator,
    EarlyStoppingCallback,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_constant_schedule,
    AdamW
)

In [5]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## ReadCsv

In [9]:
import pandas as pd
data = pd.DataFrame()
for i in range(1,6):
    temp_data = pd.read_csv(f'https://raw.githubusercontent.com/cmj-dev/groomProject/main/data/gen_train_data_{i}.csv')
    data = pd.concat([data,temp_data])
    break

In [10]:
data

,masked,original
0,쉴드 아니라 국가가해준거야.,쉴드가 아니라 국가가 면제해준거야.
1,짐승 들어ㅋㅋ,짐승입장도 들어봐야지ㅋㅋ
2,깐라고기까지,깐부라고 부르기까지 하다니
3,어머니 간 해 주기 싫어서 도망쳤다고...?,어머니 간 이식 해 주기 싫어서 도망쳤다고...?
4,##한 기념 내가 사 줄게,고생한 기념으로 내가 저녁 사 줄게
...,...,...
12334,일을 해서 숙련도가 좋겠지?,그동안 일을 꾸준히 해서 숙련도가 좋겠지?
12335,당구장에서 알바하는데 아저씨들이 나중에 밥한끼사준다면서알려달라고 그래 ; ;,당구장에서 알바하는데 아저씨들이 나중에 밥한끼사준다면서 번호알려달라고 그래;;
12336,ㅠㅠ눈빛도 완전 그윽하게 쳐다보는데 그만둘 그냥..,ㅠㅠ눈빛도 완전 그윽하게 쳐다보는데 그만둘까봐 그냥..
12337,근데니까 다른인 친구로 속이더라,근데 누구냐니까 다른 남자인 친구로 속이더라


In [11]:
data = data.sample(frac=1,random_state=43)

In [14]:
spliter = int(len(data)*0.8)

In [15]:
train_df = data.iloc[:spliter,:]
dev_df = data.iloc[spliter:,:]

In [93]:
train_df

,masked,original
12012,##면 여자 다고?,바이면 남자 여자 다 좋아한다고?
7675,사람이었어?,[UNK] 남편은 뭐하는 사람이었어?
5326,ㅋㅋ 게야,ㅋㅋ 그래도 잘생긴 게 최고야
7621,NaN,당연히 결혼해야지
8923,전쟁도 없고 질병도 어느 정도는되고 말이야,전쟁도 없고 질병도 어느 정도는 해결되고 말이야
...,...,...
11899,새이나 다름 없어!,거의 새상품이나 다름 없댔어!
2430,NaN,네가 이해해라
11265,다리에 들어가,다리에 힘이 많이 들어가네
7676,NaN,정권교체만이 답이다


In [16]:
dev_df

,masked,original
1399,수를 저 정권을 보낼 수,무슨 수를 써야 저 정권을 보낼 수 있는거야
11322,김치는 내가 먹을 모르는데 옆에서 냄새 풍기면 진짜 별로임,"김치는 내가 먹을 땐 모르는데, 옆에서 냄새 풍기면 진짜 별로임"
3281,NaN,너 미쳤어?
11309,사람 배구 선수 아냐,저 사람 그 배구 선수 아냐?
5105,조선족은 우리나라 투표 수 있나?,조선족은 우리나라 대통령 투표할 수 있나?
...,...,...
2064,애가? 울린 애가 거지,운 애가 잘못이냐? 울린 애가 잘못한 거지
10517,에이 늙 않았,에이 그정도로 늙지는 않았어
7985,##서 [UNK] 도 보이고 [UNK] 도 보이네,과거 사진에서 [UNK]도 보이고 [UNK]도 보이네
2303,##만큼는 것도,노력한만큼 돌아오는 것도 없어


In [17]:
from datasets import Dataset

train_data = Dataset.from_pandas(train_df)
dev_data = Dataset.from_pandas(dev_df)

In [47]:
train_data

Dataset({
    features: ['masked', 'original', '__index_level_0__'],
    num_rows: 9871
})

## Tokenize

In [48]:
model_name = 'skt/kogpt2-base-v2'
max_length = 256

In [49]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [74]:
tokenizer.add_special_tokens({'additional_special_tokens':['<MOR>','<CON_START>','<START>','<END>']})
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [100]:
def tokenizeWithLabel(data):
    tokenized_datas = tokenizer(
        f"<MOR> <CON_START> {data['masked']} <START> {data['original']} <END>",
        max_length=max_length,
        padding="max_length"
    )
    return tokenized_datas

In [101]:
train_tokenized_datasets = train_data.map(tokenizeWithLabel, remove_columns=train_data.column_names)
dev_tokenized_datasets = dev_data.map(tokenizeWithLabel, remove_columns=dev_data.column_names)

  0%|          | 0/9871 [00:00<?, ?ex/s]

  0%|          | 0/2468 [00:00<?, ?ex/s]